In [1]:
import pandas as pd
import pathlib
from result import Ok, Err, OkErr

# experiment_root = pathlib.Path("/Users/devjeetroy/OneDrive/ReadingStrategyExperiemntData/")
experiment_root = pathlib.Path(
    "/Users/zhangziyi/Library/CloudStorage/OneDrive-Personal/WSU-SEL-shared-all/Experiments/Experiment-Data/ReadingStrategyExperiemntData")

In [2]:
a = Ok(2)
b = a.map(lambda x: x + 1)

print(a, b)

Ok(2) Ok(3)


In [3]:
#datafiles[6]['path'].map(str).unwrap()

'/Users/devjeetroy/OneDrive/ReadingStrategyExperiemntData/p107/t1/gazeOutput-1670629314112.xml'

In [4]:
#df = pd.read_xml(
#    '/Users/devjeetroy/OneDrive/ReadingStrategyExperiemntData/p107/t1/gazeOutput-1670629314112.xml',
#    xpath='//response')

df = pd.read_xml(
    '/Users/zhangziyi/Library/CloudStorage/OneDrive-Personal/WSU-SEL-shared-all/Experiments/Experiment-Data/ReadingStrategyExperiemntData/p107/t1/gazeOutput-1670629314112.xml',
    xpath='//response')

In [5]:
df = pd.read_csv("../data/raw/eyetracking/plugin.csv")

df.dtypes

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/eyetracking/plugin.csv'

In [6]:
import numpy as np
non_numeric = df[~np.isreal(df['y'])]

In [107]:
# df.at[0, 'y'] = 'xx'
# df['y'].astype(float, errors=)
df[pd.to_numeric(df['y'], errors='coerce').isna()][['pid', 'tid', 'y']]

,pid,tid,y
1336973,p43,t3,607\nsession_end


In [9]:
import functools
from analysis.data import iter_experiment_dir, find_eyetracking_files, load_xml
from returns.result import Success, Failure
import pathlib
import pandas as pd
root = pathlib.Path(
    "/Users/devjeetroy/OneDrive/ReadingStrategyExperiemntData/")
for _, _, folder in iter_experiment_dir(root):
    result = find_eyetracking_files(folder)
    match result:
        case Success(files):
            break
        case Failure(e):
            continue

In [15]:
core = pd.DataFrame(load_xml(files['core'], xpath=".//response"))
plugin = pd.DataFrame(load_xml(files['plugin'], xpath=".//response"))

print(core.columns)

Index(['event_id', 'core_time', 'tracker_time', 'x', 'y', 'left_x', 'left_y',
       'left_pupil_diameter', 'left_validation', 'right_x', 'right_y',
       'right_pupil_diameter', 'right_validation', 'user_left_x',
       'user_left_y', 'user_left_z', 'user_right_x', 'user_right_y',
       'user_right_z'],
      dtype='object')


In [20]:
import pandera as pa
common_schema = {
    "x": pa.Column(pa.Float),
    "y": pa.Column(pa.Float),
    "event_id": pa.Column(pa.Int),
}
itrace_core_shema = pa.DataFrameSchema({
    **common_schema,
    'core_time': pa.Column(pa.Int),
    'tracker_time': pa.Column(pa.Int),
    'left_x': pa.Column(pa.Float),
    'right_x': pa.Column(pa.Float),
    'left_y': pa.Column(pa.Float),
    'right_y': pa.Column(pa.Float),
    'left_pupil_diameter': pa.Column(pa.Float),
    'right_pupil_diameter': pa.Column(pa.Float),
    'left_validation': pa.Column(pa.Int),
    'right_validation': pa.Column(pa.Int),
    'user_left_x': pa.Column(pa.Float),
    'user_left_y': pa.Column(pa.Float),
    'user_left_z': pa.Column(pa.Float),
    'user_right_x': pa.Column(pa.Float),
    'user_right_y': pa.Column(pa.Float),
    'user_right_z': pa.Column(pa.Float),
}, coerce=True)
itrace_atom_schema = pa.DataFrameSchema({
    **common_schema,
    "plugin_time": pa.Column(pa.Float),
    "gaze_target": pa.Column(pa.Category),
    "gaze_target_type": pa.Column(pa.Category),
    "source_file_path": pa.Column(pa.Category),
    "source_file_line": pa.Column(pa.Int),
    "source_file_col": pa.Column(pa.Int),
    "editor_line_height": pa.Column(pa.Float, nullable=True),
    "editor_font_height": pa.Column(pa.Float, nullable=True),
    # "editor_line_base_x": pa.Column(pa.Float, nullable=True),
    # "editor_line_base_y": pa.Column(pa.Float, nullable=True),
}, coerce=True)



itrace_atom_schema.validate(plugin).dtypes
itrace_core_shema.validate(core).dtypes

event_id                  int64
core_time                 int64
tracker_time              int64
x                       float64
y                       float64
left_x                  float64
left_y                  float64
left_pupil_diameter     float64
left_validation           int64
right_x                 float64
right_y                 float64
right_pupil_diameter    float64
right_validation          int64
user_left_x             float64
user_left_y             float64
user_left_z             float64
user_right_x            float64
user_right_y            float64
user_right_z            float64
dtype: object

In [36]:
core = pd.read_parquet("../data/raw/eyetracking/core.parq")
plugin = pd.read_parquet("../data/raw/eyetracking/plugin.parq")

core = core.astype({'pid': 'category', 'tid': 'category'})
plugin = plugin.astype({k: 'category' for k in ['pid', 'tid', 'gaze_target_type', 'source_file_path', 'gaze_target']})

In [67]:
fixations = pd.read_parquet("../data/processed/fixations.parq")

In [72]:
fixations.query(f"pid == 'p10' & tid == 't1'")

,,duration,n,plugin_time,source_file_line,source_file_col,pid,tid,gaze_target,left_pupil_diameter,right_pupil_diameter,x,y
61,0,116.0,15.0,1.636745e+12,282.0,49.0,p10,t1,graph_utils_c.cc,14.546060,18.455841,650.666667,444.733333
183,0,93.0,10.0,1.636745e+12,282.0,19.0,p10,t1,graph_utils_c.cc,15.337935,19.119728,369.700000,458.300000
231,0,107.0,13.0,1.636745e+12,282.0,8.0,p10,t1,graph_utils_c.cc,14.774688,17.352651,256.538462,425.461538
319,0,94.0,13.0,1.636745e+12,280.0,9.0,p10,t1,graph_utils_c.cc,17.107777,17.817830,249.846154,381.615385
365,0,90.0,11.0,1.636745e+12,275.0,63.0,p10,t1,graph_utils_c.cc,14.571436,15.799448,808.090909,241.454545
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11257,0,85.0,12.0,1.636745e+12,286.0,20.0,p10,t1,graph_utils_c.cc,15.559627,17.788493,359.166667,694.250000
11265,0,118.0,14.0,1.636745e+12,287.0,20.0,p10,t1,graph_utils_c.cc,15.462895,17.647549,369.071429,681.214286
11287,0,190.0,25.0,1.636745e+12,287.0,33.0,p10,t1,graph_utils_c.cc,15.720620,18.175006,492.280000,698.640000
11301,0,158.0,20.0,1.636745e+12,289.0,28.0,p10,t1,graph_utils_c.cc,15.759643,17.771248,439.950000,720.300000


In [71]:
original = root / 'p10' / 't1' / 'processed-new' / 'fixations.json'
df = pd.read_json(original)

df

,duration,n,system_time,source_file_line,source_file_col,target,left_pupil_diameter,right_pupil_diameter,x,y
0,116,15,2021-11-12 19:20:06.224999936,283,49,graph_utils_c.cc,14.546060,18.455841,650.666667,444.733333
1,93,10,2021-11-12 19:20:08.527000064,283,19,graph_utils_c.cc,15.337935,19.119728,369.700000,458.300000
2,107,13,2021-11-12 19:20:09.580000000,283,8,graph_utils_c.cc,14.774688,17.352651,256.538462,425.461538
3,94,13,2021-11-12 19:20:11.468000000,281,9,graph_utils_c.cc,17.107777,17.817830,249.846154,381.615385
4,90,11,2021-11-12 19:20:13.803000064,276,63,graph_utils_c.cc,14.571436,15.799448,808.090909,241.454545
...,...,...,...,...,...,...,...,...,...,...
624,85,12,2021-11-12 19:26:11.300000000,287,20,graph_utils_c.cc,15.559627,17.788493,359.166667,694.250000
625,118,14,2021-11-12 19:26:11.575000064,288,20,graph_utils_c.cc,15.462895,17.647549,369.071429,681.214286
626,190,25,2021-11-12 19:26:12.105999872,288,33,graph_utils_c.cc,15.720620,18.175006,492.280000,698.640000
627,158,20,2021-11-12 19:26:12.516000000,290,28,graph_utils_c.cc,15.759642,17.771248,439.950000,720.300000


## Interim

In [65]:
import numpy as np

n = 10
df = pd.DataFrame(
    {
        "A": np.random.randint(1, 20, n),
        "B": np.random.randint(20, 40, n),
        "C": [True] * (n // 2) + [False] * (n // 2),
    }
)

df


,A,B,C
0,5,22,True
1,12,39,True
2,13,36,True
3,19,23,True
4,16,39,True
5,18,39,False
6,5,23,False
7,5,38,False
8,19,35,False
9,8,20,False


In [66]:
def agger(df):
    print(df.head())
    return df

df.groupby('C').agg(agger)

5    18
6     5
7     5
8    19
9     8
Name: A, dtype: int64
5    39
6    23
7    38
8    35
9    20
Name: B, dtype: int64
    A   B
5  18  39
6   5  23
7   5  38
8  19  35
9   8  20
    A   B
0   5  22
1  12  39
2  13  36
3  19  23
4  16  39


ValueError: Data must be 1-dimensional

In [18]:
print(len(core))

print(len(plugin))
shared_columns = core.columns.intersection(plugin.columns)
eyetracking_df = pd.merge(core, plugin, on=shared_columns.tolist(), how='inner')

assert len(eyetracking_df.columns) == len(set(eyetracking_df.columns))

188869
128691


In [19]:
eyetracking_df[['pid', 'tid']].drop_duplicates()

,pid,tid
0,p100,t2
19281,p100,t4
37893,p100,t3
76128,p107,t1


In [21]:
core[['pid', 'tid']].drop_duplicates()

,pid,tid
0,p100,t2
0,p100,t4
0,p100,t1
0,p100,t3
0,p107,t1
